# 数据挖掘

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sns
from datetime import datetime, timedelta

## 数据转换

- 数据挖掘（机器学习）前，数据需要先过滤、整理、转换。
- 在[数据探索](try-exploration.ipynb)阶段，发现[测试数据](test_data.xls)某些字段需要额外处理。

字段 | 类型 | 处理方式
---|---|---
ship_name | 杂讯 | 丢弃
ship_mobile | 分类变量 | 转换成 level
order_id | 无关分析 | 丢弃
total_amount | 数值 | 直接使用
final_amount | 信息重复 | 丢弃
pay_status | 分类变量 | 转换成 level
ship_status | 分类变量 | 直接使用 (已编码)
is_delivery | 无识别度 | 丢弃
try_order | 无识别度 | 丢弃
erp_order_id | 分类变量 | 转换成 level
erp_status | 分类变量 | 转换成 level
createtime | 数值 | 转换成 日期
last_modified | 数值 | 转换成 日期
payment | 分类变量 | 转换成 level
shipping_id | 无识别度 | 丢弃
shipping | 分类变量 | 转换成 level
member_id | 分类变量 | 转换成 level
promotion_type | 分类变量 | 转换成 level
group_id | 分类变量 | 转换成 level
groupOn_id | 分类变量 | 转换成 level
is_leader | 分类变量 | 直接使用 (已编码)
is_prepare | 无识别度 | 丢弃
group_num | 分类变量 | 转换成 level
status | 分类变量 | 转换成 level
confirm_delivery | 分类变量 | 转换成 level
confirm | 无识别度 | 丢弃
ship_area | 分类变量 | 转换成 level
weight | 分类变量 | 转换成 level
tostr | 无关分析 | 丢弃
itemnum | 分类变量 | 转换成 level
ip | 杂讯 | 丢弃
ship_addr | 信息重复 | 丢弃
ship_zip | 无关分析 | 丢弃
ship_tel | 信息重复 | 丢弃
ship_email | 无识别度 | 丢弃
ship_time | 分类变量 | 转换成 level
cost_item | 数值 | 直接使用
is_tax | 布林值 | 直接使用
tax_type | 分类变量 | 转换成 level
cost_tax | 无识别度 | 丢弃
is_protect | 无识别度 | 丢弃
cost_protect | 无识别度 | 丢弃
cost_payment | 无识别度 | 丢弃
currency | 无识别度 | 丢弃
cur_rate | 无识别度 | 丢弃
advance | 数值 | 直接使用
score_u | 数值 | 直接使用
score_g | 数值 | 直接使用
discount | 数值 | 直接使用
pmt_goods | 数值 | 直接使用
pmt_order | 数值 | 直接使用
payed | 杂讯 | 丢弃
disabled | 无识别度 | 丢弃
displayonsite | 布林值 | 直接使用
mark_type | 无识别度 | 丢弃
cost_freight | 数值 | 直接使用
extend | 无识别度 | 丢弃
order_refer | 无识别度 | 丢弃
addon | 杂讯 | 丢弃
source | 分类变量 | 转换成 level
city_link | 分类变量 | 直接使用 (已编码)
verify_city_link | 分类变量 | 直接使用 (已编码)
fuzzy | 数值 | 直接使用
longitude | 数值 | 直接使用
latitude | 数值 | 直接使用
scalping | 布林值 | 直接使用
out_trade_no | 无关分析 | 丢弃
autoSendErp | 布林值 | 直接使用
autoSendErpStatus | 分类变量 | 直接使用 (已编码)
string_tag_id | 杂讯 | 丢弃
lang | 分类变量 | 转换成 level
delivery_note_price | 分类变量 | 转换成 level

### 步骤一：过滤字段

In [3]:
# 选取需要的字段
cols = ['ship_mobile', 'total_amount', 'pay_status', 'ship_status', 'erp_order_id', 'erp_status', 'createtime', 'last_modified', 'payment', 'shipping', 'member_id', 'promotion_type', 'group_id', 'groupOn_id', 'is_leader', 'group_num', 'status', 'confirm_delivery', 'ship_area', 'weight', 'itemnum', 'ship_time', 'cost_item', 'is_tax', 'tax_type', 'advance', 'score_u', 'score_g', 'discount', 'pmt_goods', 'pmt_order', 'displayonsite', 'cost_freight', 'source', 'city_link', 'verify_city_link', 'fuzzy', 'longitude', 'latitude', 'scalping', 'autoSendErp', 'autoSendErpStatus', 'lang', 'delivery_note_price']
df = pd.read_excel('test_data.xls', usecols=cols)

### 步骤二：转换格式

In [4]:
# 函数：转换level，重新编码
def levelize(values):
    items = values.unique()
    mapping = dict((item, idx) for idx, item in enumerate(items))
    return values.map(mapping)

In [5]:
df.ship_mobile = levelize(df.ship_mobile)

In [6]:
df.pay_status = levelize(df.pay_status)

In [7]:
df.erp_order_id = levelize(df.erp_order_id)

In [8]:
df.erp_status = df.erp_status.map({'succ': 0, 'normal': 1})

In [9]:
df.createtime = df.createtime.map(datetime.fromtimestamp).map(lambda ts: ts.dayofyear)

In [10]:
df.last_modified = df.last_modified.map(datetime.fromtimestamp).map(lambda ts: ts.dayofyear)

In [11]:
df.payment = df.payment.map({'wxpayjsapi': 0, 'online': 1, 'alipay': 2, 'wxsmpay': 3, '-1': 4})

In [12]:
df.shipping = df.shipping.map({u'冷链直供': 0, u'一米市集直送': 1})

In [13]:
df.member_id = levelize(df.member_id)

In [14]:
df.promotion_type = df.promotion_type.map({'normal': 0, 'group': 1, 'prepare': 2, 'card': 3})

In [15]:
df.group_id = levelize(df.group_id)

In [16]:
df.groupOn_id = levelize(df.groupOn_id)

In [17]:
df.group_num = levelize(df.group_num)

In [18]:
df.status = df.status.map({'finish': 0, 'dead': 1, 'active': 2, 'pendding': 3})

In [19]:
df.confirm_delivery = df.confirm_delivery.map({'Y': 0, 'N': 1})

In [20]:
df.ship_area = df.ship_area.map(lambda area: int(area.split(':')[-1]))

In [21]:
def time2code(t):
    mapping = {
        '08:00-12:00': 0,
        '14:00-18:00': 1,
        '18:00-21:00': 2,
        '14:00-21:00': 3}
    return mapping.get(t, 4)

df.ship_time = df.ship_time.fillna('??,??').map(lambda x: time2code(x.split(',')[1]))

In [22]:
# df.is_tax = df.is_tax.map({False: 0, True: 1})

In [23]:
df.tax_type = df.tax_type.map({'false': 0, 'company': 1, 'personal': 2})

In [24]:
# df.displayonsite = df.displayonsite.map({False: 0, True: 1})

In [25]:
df.source = df.source.map({'weixin': 0, 'ios': 1, 'pc': 2, 'wap': 3})

In [26]:
df.scalping = df.scalping.map({False: 0, True: 1})

In [27]:
# df.autoSendErp = df.autoSendErp.map({False: 0, True: 1})

In [28]:
df.lang = df.lang.map({'simplified_chinese': 0, 'english': 1})

In [29]:
df.delivery_note_price = df.delivery_note_price.map({'display': 0, 'none': 1})

In [30]:
df.ix[0]

ship_mobile                  0
total_amount                34
pay_status                   0
ship_status                  1
erp_order_id                 0
erp_status                   0
createtime                   2
last_modified                8
payment                      0
shipping                     0
member_id                    0
promotion_type               0
group_id                     0
groupOn_id                   0
is_leader                    0
group_num                    0
status                       0
confirm_delivery             0
ship_area                   35
weight                       6
itemnum                      1
ship_time                    4
cost_item                   36
is_tax                   False
tax_type                     0
advance                      0
score_u                    200
score_g                     36
discount                     0
pmt_goods                    0
pmt_order                    0
displayonsite             True
cost_fre

### 步骤三：去除异常值

根据[箱形图](http://baike.baidu.com/link?url=p5iilyxKfj00v8yBe7AkHusrO94ulEySuxDyqefi_6Qk34eSjJDe0MYvF2ZWFY1ke_nRy4aI8svvxKttWPgD1ti0vL7DW6nTGeFwlVlnwDKzFojUsp32aaQ6km01wIa9Rps-Va0ntNji-_kvHnNHzA_d3XTX-bpVspI3_jiFOM0CqZe0767L0SCo9qr9SyOR)定义，过滤极端异常值。

In [31]:
# 定义异常阈值
outlier_thershold = 5

# 函数：选出异常值
def choose_abnormal(values):
    Q1 = values.quantile(.25)
    Q3 = values.quantile(.75)
    IQR = Q3 - Q1
    upper_fence = Q3 + outlier_thershold * IQR
    lower_fence = Q1 - outlier_thershold * IQR
    return (values < lower_fence) | (values > upper_fence)

In [32]:
# 过滤订购金额异常值
abnormals = choose_abnormal(df.total_amount)
normals = abnormals.map(lambda x: not x)
print "toatl: %d, abnormal: %d" % (len(df[normals]), len(df[abnormals]))

df = df[normals]

toatl: 5867, abnormal: 14


In [33]:
# 过滤地理位置异常值
log_abnormals = choose_abnormal(df.longitude) # 经度异常值
lat_abnormals = choose_abnormal(df.latitude)  # 纬度异常值
abnormals = log_abnormals | lat_abnormals     # 经纬度异常值
normals = abnormals.map(lambda x: not x)      # 经纬度正常值
print "toatl: %d, abnormal: %d" % (len(df[normals]), len(df[abnormals]))

df = df[normals]

toatl: 5852, abnormal: 15
